In [1]:
import pandas as pd

In [3]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, EasterMonday
from pandas.tseries.offsets import DateOffset, Day
import datetime

### Упражнение 1

* Скачайте с портала Открытых Данных производственный календарь России: https://data.gov.ru/opendata/7708660670-proizvcalendar
* На его основе определите соответствующий календарь в Pandas: `RussianBusinessCalendar(AbstractHolidayCalendar)`
* С помощью полученного в пункте выше календаря и модуля `pd.offsets` создайте DataFrame one-hot календарных признаков:
    * День недели
    * Месяц
    * Выходной день
    * Праздничный день
    * Предпраздничный рабочий день
    * Последний день месяца
    * Последний рабочий день месяца
    * Предновогодний выходной день

In [1]:
# пример требуемого DataFrame 
events

,выходной,праздник,предпраздничный рабочий день,последний день месяца,последний рабочий день месяца,пн,вт,ср,чт,пт,сб,вс,январь,февраль,март,апрель,май,июнь,июль,август,сентябрь,октябрь,ноябрь,декабрь,Предновогодний выходной день
2014-01-01,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-02,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-03,1,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-04,1,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2014-01-05,1,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


### Упражнение 2

* Для временного ряда курса акций Сбербанка на основе производственного календаря России сформируйте DataFrame признаков:
    * Лаговые переменные рабочих дней с глубинами: 1, 3, 5
    * Скользящие статистики `['mean', 'median', 'max', 'min', 'std'] ` с окнами 1, 3, 5 сгрупированные по дням недели
    * Статистики `['mean', 'median', 'max', 'min', 'std'] ` в расширяющемся окне сгруппированные по рабочим, праздничным и предпраздничным дням

In [158]:
df = pd.read_csv('./data/data-20191112T1252-structure-20191112T1247.csv')
df1 = pd.read_csv('./data/data-20191112T1252-structure-20191112T1247.csv', names=[i for i in range(18)], usecols=[i for i in range(13)], index_col=0, header=0)
#                  usecols=[ 'Год/Месяц', 'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'],
#                  index_col='Год/Месяц')
df1.head(1)

,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,,,,,,,,,
1999,"1,2,3,4,6*,7,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,24,25,30*","1,2,3,4,8,9,10,15,16,22,23,29,30","5,6,11*,12,13,14,19,20,26,27","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31","6,7,8,13,14,20,21,27,28","4,5,11,12,13,18,19,25,26,31*"


In [159]:
df1.tail(6)

,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,,,,,,,,,
2020,"1,2,3,4,5,6,7,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,24+,29","1,7,8,9+,14,15,21,22,28,29","4,5,11,12,18,19,25,26,30*","1,2,3,4+,5+,8*,9,10,11+,16,17,23,24,30,31","6,7,11*,12,13,14,20,21,27,28","4,5,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","3,4,10,11,17,18,24,25,31","1,3*,4,7,8,14,15,21,22,28,29","5,6,12,13,19,20,26,27,31*"
2021,"1,2,3,4,5,6,7,8,9,10,16,17,23,24,30,31","6,7,13,14,20,21,22*,23,27,28","6,7,8*,13,14,20,21,27,28","3,4,10,11,17,18,24,25,30*","1,2,3+,8,9,10+,15,16,22,23,29,30","5,6,11*,12,13,14+,19,20,26,27","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","4,5,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31","3*,4,6,7,13,14,20,21,27,28","4,5,11,12,18,19,25,26,31*"
2022,"1,2,3,4,5,6,7,8,9,15,16,22,23,29,30","5,6,12,13,19,20,22*,23,26,27","5,6,7*,8,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30","1,2+,7,8,9,14,15,21,22,28,29","4,5,11,12,13+,18,19,25,26","2,3,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","3,4,10,11,17,18,24,25","1,2,8,9,15,16,22,23,29,30","3*,4,5,6,12,13,19,20,26,27","3,4,10,11,17,18,24,25,31"
2023,"1,2,3,4,5,6,7,8,14,15,21,22,28,29","4,5,11,12,18,19,22*,23,25,26","4,5,7*,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,29,30","1,6,7,8*,9,13,14,20,21,27,28","3,4,10,11,12,17,18,24,25","1,2,8,9,15,16,22,23,29,30","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30","1,7,8,14,15,21,22,28,29","3*,4,5,6+,11,12,18,19,25,26","2,3,9,10,16,17,23,24,30,31"
2024,"1,2,3,4,5,6,7,8,13,14,20,21,27,28","3,4,10,11,17,18,22*,23,24,25","2,3,7*,8,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28,30*","1,4,5,8*,9,11,12,18,19,25,26","1,2,8,9,11*,12,15,16,22,23,29,30","6,7,13,14,20,21,27,28","3,4,10,11,17,18,24,25,31","1,7,8,14,15,21,22,28,29","5,6,12,13,19,20,26,27","2,3,4,9,10,16,17,23,24,30","1,7,8,14,15,21,22,28,29,31*"
2025,"1,2,3,4,5,6,7,8,11,12,18,19,25,26","1,2,8,9,15,16,22,23,24+","1,2,7*,8,9,10+,15,16,22,23,29,30","5,6,12,13,19,20,26,27,30*","1,3,4,8*,9,10,11,17,18,24,25,31","1,7,8,11*,12,14,15,21,22,28,29","5,6,12,13,19,20,26,27","2,3,9,10,16,17,23,24,30,31","6,7,13,14,20,21,27,28","4,5,11,12,18,19,25,26","1,2,3*,4,8,9,15,16,22,23,29,30","6,7,13,14,20,21,27,28,31*"


0
1,2,3,4,6*,7,9,10,16,17,23,24,30,31
6,7,13,14,20,21,27,28
6,7,8,13,14,20,21,27,28
3,4,10,11,17,18,24,25,30*
1,2,3,4,8,9,10,15,16,22,23,29,30
5,6,11*,12,13,14,19,20,26,27
3,4,10,11,17,18,24,25,31
1,7,8,14,15,21,22,28,29
4,5,11,12,18,19,25,26
2,3,9,10,16,17,23,24,30,31
6,7,8,13,14,20,21,27,28
4,5,11,12,13,18,19,25,26,31*
1
1,2,3,4,6*,7,8,9,15,16,22,23,29,30
5,6,12,13,19,20,26,27
4,5,7*,8,11,12,18,19,25,26
1,2,8,9,15,16,22,23,29,30
1,2,6,7,8*,9,13,14,20,21,27,28
3,4,10,11,12,17,18,24,25
1,2,8,9,15,16,22,23,29,30
5,6,12,13,19,20,26,27
2,3,9,10,16,17,23,24,30
1,7,8,14,15,21,22,28,29
4,5,7,11,12,18,19,25,26
2,3,9,10,11*,12,16,17,23,24,30,31
2
1,2,6,7,8,13,14,20,21,27,28
3,4,10,11,17,18,24,25
3,4,7*,8,10,11,17,18,24,25,31
1,7,8,14,15,21,22,28,29,30*
1,2,5,6,8*,9,12,13,19,20,26,27
2,3,9,10,11*,12,16,17,23,24,30
1,7,8,14,15,21,22,28,29
4,5,11,12,18,19,25,26
1,2,8,9,15,16,22,23,29,30
6,7,13,14,20,21,27,28
3,4,6*,7,10,11,17,18,24,25
1,2,8,9,12,15,16,22,23,29,30
3
1,2,5,6,7,12,13,19,20,26,27
2,3

In [154]:
df1 = pd.DataFrame(df.head(1))
print(df.loc[0]['Январь'])
print(df.loc[0]['Январь'].partition('*')[0][-1])

1,2,3,4,6*,7,9,10,16,17,23,24,30,31
6


In [ ]:
rules = [Holiday(name='Выходной или праздник', month=3, day=6, year=2020,
                     offset=Day(1)) for ]

In [60]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
    start_date = datetime.datetime(1999, 1, 1)
    end_date = datetime.datetime(2025, 12, 31)
#     rules = [Holiday('Рабочая суббота', month=3, day=6, year=2020,
#                      offset=Day(1))]
    rules = [Holiday(name='Выходной или праздник', month=3, day=6, year=2020,
                     offset=Day(1))]
    
russian_busday = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri', 
                                              calendar=RussianBusinessCalendar())
dt = datetime.datetime(2021, 3, 5)
print(dt + russian_busday)

2021-03-08 00:00:00


()

In [40]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
    start_date = datetime.datetime(1999, 1, 1)
    end_date = datetime.datetime(2025, 12, 31)
#     rules = [Holiday('Рабочая суббота', month=3, day=6, year=2020,
#                      offset=Day(1))]
    rules = [Holiday("July 3rd", month=7, day=3, days_of_week=(0, 1, 2, 3))]
    
russian_busday = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri', 
                                              calendar=RussianBusinessCalendar())
dt = datetime.datetime(2020, 3, 4)
print(dt + russian_busday)

2020-03-05 00:00:00
